In [ ]:
!pip install scikit-fuzzy

In [ ]:
# ===========================================
# SISTEMA EXPERTO DIFUSO - DETECCIÓN DE TERREMOTOS
# Versión completa (sin zonas muertas)
# ===========================================

!pip install -q scikit-fuzzy

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# --- Definición de variables ---
aceleracion = ctrl.Antecedent(np.arange(0, 1.1, 0.01), 'aceleracion')   # g (0–1)
frecuencia = ctrl.Antecedent(np.arange(0, 20.1, 0.1), 'frecuencia')     # Hz (0–20)
duracion = ctrl.Antecedent(np.arange(0, 60.1, 1), 'duracion')           # s (0–60)
amenaza = ctrl.Consequent(np.arange(0, 11, 1), 'amenaza')               # Nivel (0–10)

# --- Funciones de pertenencia ---
aceleracion['baja'] = fuzz.trimf(aceleracion.universe, [0, 0, 0.3])
aceleracion['media'] = fuzz.trimf(aceleracion.universe, [0.2, 0.5, 0.8])
aceleracion['alta'] = fuzz.trimf(aceleracion.universe, [0.6, 1, 1])

frecuencia['baja'] = fuzz.trapmf(frecuencia.universe, [0, 0, 2, 5])
frecuencia['media'] = fuzz.trapmf(frecuencia.universe, [3, 6, 10, 13])
frecuencia['alta'] = fuzz.trapmf(frecuencia.universe, [11, 15, 20, 20])

duracion['corta'] = fuzz.trimf(duracion.universe, [0, 0, 10])
duracion['media'] = fuzz.trimf(duracion.universe, [5, 20, 35])
duracion['larga'] = fuzz.trimf(duracion.universe, [30, 60, 60])

amenaza['sin_actividad'] = fuzz.trimf(amenaza.universe, [0, 0, 2])
amenaza['microtemblor'] = fuzz.trimf(amenaza.universe, [1, 3, 5])
amenaza['leve'] = fuzz.trimf(amenaza.universe, [4, 5, 6])
amenaza['moderado'] = fuzz.trimf(amenaza.universe, [5, 7, 8])
amenaza['fuerte'] = fuzz.trimf(amenaza.universe, [7, 9, 10])
amenaza['destructivo'] = fuzz.trimf(amenaza.universe, [9, 10, 10])

# --- Reglas completas (27 combinaciones) ---
rules = []

# Para cada combinación de aceleración / frecuencia / duración
niveles_acel = ['baja', 'media', 'alta']
niveles_freq = ['baja', 'media', 'alta']
niveles_dur = ['corta', 'media', 'larga']

# Lógica heurística: cuanto mayor la aceleración, menor frecuencia y mayor duración => más destructivo
for a in niveles_acel:
    for f in niveles_freq:
        for d in niveles_dur:
            # Base heurística: ponderar impacto
            if a == 'baja' and f == 'alta' and d == 'corta':
                out = 'sin_actividad'
            elif a == 'baja':
                out = 'microtemblor'
            elif a == 'media' and f == 'alta':
                out = 'leve'
            elif a == 'media' and f == 'media' and d != 'corta':
                out = 'moderado'
            elif a == 'media':
                out = 'leve'
            elif a == 'alta' and f == 'alta' and d == 'corta':
                out = 'microtemblor'
            elif a == 'alta' and f == 'media' and d == 'corta':
                out = 'leve'
            elif a == 'alta' and f == 'baja' and d == 'larga':
                out = 'destructivo'
            elif a == 'alta' and f == 'baja':
                out = 'fuerte'
            else:
                out = 'moderado'

            rules.append(ctrl.Rule(aceleracion[a] & frecuencia[f] & duracion[d], amenaza[out]))

# Crear sistema de control
control_sismo = ctrl.ControlSystem(rules)
simulador = ctrl.ControlSystemSimulation(control_sismo)

# --- Entrada por terminal (funciona en Colab si está en celda individual) ---
try:
    valor_acel = float(input("Ingrese la aceleración del terreno (g, 0–1): "))
    valor_freq = float(input("Ingrese la frecuencia dominante (Hz, 0–20): "))
    valor_dur = float(input("Ingrese la duración del evento (s, 0–60): "))
except Exception:
    print("❌ Error de entrada. Reinicia la celda e ingresa valores numéricos válidos.")
    raise SystemExit(1)

# Asignar y calcular
simulador.input['aceleracion'] = valor_acel
simulador.input['frecuencia'] = valor_freq
simulador.input['duracion'] = valor_dur
simulador.compute()

# --- Mostrar resultados ---
nivel_amenaza = simulador.output['amenaza']
print(f"\n🔍 Nivel de amenaza sísmica (defuzzificado): {nivel_amenaza:.2f} / 10")

if nivel_amenaza < 2:
    print("➡️ Clasificación: Sin actividad sísmica.")
elif nivel_amenaza < 4:
    print("➡️ Clasificación: Microtemblor.")
elif nivel_amenaza < 6:
    print("➡️ Clasificación: Terremoto leve.")
elif nivel_amenaza < 8:
    print("➡️ Clasificación: Terremoto moderado.")
elif nivel_amenaza < 9.5:
    print("➡️ Clasificación: Terremoto fuerte.")
else:
    print("➡️ Clasificación: Terremoto destructivo.")

# --- Gráficas ---
plt.figure(figsize=(10, 6))
aceleracion.view()
frecuencia.view()
duracion.view()
amenaza.view(simulador)
plt.show()
